In [17]:
import requests
import json
import os
import io
import zipfile
import shapefile
from shapely.geometry import Point, shape
from shapely import geometry
import pandas as pd
import fiona
from fiona.crs import from_epsg
import datetime
import random

In [18]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab4')
wksp = os.getcwd()

## Pull weather information

In [19]:
# Get the 153 weather stations of the Minnesota network selecting a day when data were reported for all of them
link = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?date=2023-03-15&network=MN_RWIS'
info = json.loads(requests.get(link).text)
location = []
for i in range(len(info['features'])):
    # Store the station and its coordinates
    location.append({'station': info['features'][i]['properties']['station'], 
                     'coordinates': info['features'][i]['geometry']['coordinates']})

In [20]:
# Daily weather data from 2023
url = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2023'
weather = json.loads(requests.get(url).text)

In [21]:
# Delete all the unneeded weather variables and keep only minimum and maximum temperature
delete = [
      "tmpf_est",
      "precip",
      "precip_est",
      "max_gust",
      "snow",
      "snowd",
      "min_rh",
      "max_rh",
      "min_dwpf",
      "max_dwpf",
      "min_feel",
      "max_feel",
      "min_rstage",
      "max_rstage",
      "temp_hour",
      "max_gust_localts",
      "max_drct",
      "avg_feel", 
      "avg_sknt", 
      "vector_avg_drct", 
      "id"
]

for i in range(len(weather['features'])):
    for key in delete:
        del weather['features'][i]['properties'][key]

## QA

In [22]:
# Read in the shapefile data for Minnesota
sf = shapefile.Reader("minnesota.shp")
shapes = sf.shapes()
state_border = shapes[0]

# Create a shapely Polygon object from the state border shape
border_polygon = shape(state_border)

# Create an empty list to add the not useful readings
wrong = []

# Add all the not useful readings to a list
for i in range(len(weather['features'])):
    
    # Readings whose temp readings are None
    if weather['features'][i]['properties']['min_tmpf'] == None or weather['features'][i]['properties']['max_tmpf'] == None:
        wrong.append(weather['features'][i])
        continue
    
    # Stations outside of Minnesota
    point = Point(weather['features'][i]['geometry']['coordinates'])
    if not border_polygon.contains(point):
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose min and max temp are the same. This is an indicator of wrong data
    if weather['features'][i]['properties']['min_tmpf'] == weather['features'][i]['properties']['max_tmpf']:
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose temp is outside of the optimum range
    lower_limit = math.floor(weather['features'][i]['properties']['min_tmpf'])
    upper_limit = math.ceil(weather['features'][i]['properties']['max_tmpf'])
    range_temp = range(lower_limit, upper_limit)
    
    # Readings not representative of the broader region if max and min temp are similar
    if len(range_temp) == 1:
        wrong.append(weather['features'][i]) 
        continue
    
# Delete the not useful readings 
for element in wrong:
    weather['features'].remove(element)

## Monthly average temperature

In [23]:
stations = []
# Add the dictionaries to a data frame
for j in range(len(weather['features'])):
    stations.append(weather['features'][j]['properties'])
df = pd.DataFrame.from_dict(stations)

# Remove the day part from the date leaving only year and month
for i in range(len(df['date'])):
    df['date'][i] = df['date'][i][:7]
    
# Get monthly average min and max temperature for each station
grouped = df.groupby(['station', 'date', 'name']).agg('mean')
grouped.reset_index(inplace=True)

# Return data to a dictionary
mean = grouped.to_dict('records')

[23]:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
## Add the geometry to the stations
mean_tmp = []
for i in range(len(mean)):
    for j in range(len(location)):
        if mean[i]['station'] == location[j]['station']:
            mean_tmp.append({'type': 'Feature', 'properties': mean[i], 
                             'geometry': {'type': 'Point', 'coordinates': location[j]['coordinates']}})

## Stations shapefile

In [25]:
schema =  {'geometry': 'Point', 'properties': {'station': 'str', 'date': 'str', 'name': 'str', 'max_tmpf': 'float', 'min_tmpf': 'float'}}

with fiona.open("stations.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = schema) as output:
    for i in range(len(mean_tmp)):
          # geometry
          point = Point(mean_tmp[i]['geometry']['coordinates'])
          # attributes
          prop = mean_tmp[i]['properties']
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})

In [26]:
now = datetime.datetime.now()
current_year_month = now.strftime("%Y-%m")

# Extract only the current month's min temp
arcpy.analysis.Select(
    in_features="stations.shp",
    out_feature_class="current_temp.shp",
    where_clause=f"date = '{current_year_month}'"
)

<Result 'E:\\ArcGIS_2\\Lab4\\current_temp.shp'>

## Interpolation

In [27]:
# Interpolate min temperature using Kriging

desc = arcpy.Describe('minnesota.shp')

with arcpy.EnvManager(extent=desc.extent):
    arcpy.ddd.Kriging(
        in_point_features="current_temp.shp",
        z_field="min_tmpf",
        out_surface_raster=os.path.join(wksp, "Kriging_min.tif"),
        semiVariogram_props="Spherical 0.023537 # # #",
        cell_size=0.1,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )

## Append Interpolated Temperature Values to City Data

In [28]:
arcpy.FeatureToPoint_management("cities.shp", "cities_centroids.shp", "CENTROID")

<Result 'E:\\ArcGIS_2\\Lab4\\cities_centroids.shp'>

In [29]:
arcpy.sa.ExtractValuesToPoints(
    in_point_features="cities_centroids.shp",
    in_raster="Kriging_min.tif", 
    out_point_features=r"cities_min_temp.shp",
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY"
)

<geoprocessing server result object object at 0x0000023E850982D0>

In [30]:
# Rename field
arcpy.management.CalculateField(
        in_table="cities_min_temp.shp",
        field="min_temp",
        expression="!RASTERVALU!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.DeleteField(
    in_table="cities_min_temp.shp",
    drop_field="ORIG_FID;RASTERVALU",
    method="DELETE_FIELDS"
)

<Result 'cities_min_temp.shp'>